In [1]:
import pandas as pd
import os, glob, re, shutil
import PIL.ExifTags as ExifTags
import PIL.Image as Image
from tqdm import tqdm
from datetime import datetime
from typing import List, Tuple, Union
import piexif

# Naming conventions

In [4]:
def get_exif(image_path: Union[str, ]) -> dict:
    with Image.open(image_path) as image:
        exif_data = image._getexif()
        if not exif_data:
            width, height = image.size
            return {'ExifImageWidth': width, 'ExifImageHeight': height}

        exif = {}
        for tag, value in exif_data.items():
            decoded = ExifTags.TAGS.get(tag, tag)
            exif[decoded] = value
        return exif

def get_exif_gps(exif_info: dict):
    gps_info = {}

    for key in exif_info['GPSInfo'].keys():
        decode = ExifTags.GPSTAGS.get(key,key)
        gps_info[decode] = exif_info['GPSInfo'][key]#int.from_bytes(exif_info['GPSInfo'][key], byteorder='big')
    return gps_info

def get_exif_orientation(img_path: str):
    """
    In EXIF at id 274 hexadecimal 0x0112, there is a tag called Orientation
    1 = Horizontal (normal)
    2 = Mirror horizontal
    3 = Rotate 180
    4 = Mirror vertical
    5 = Mirror horizontal and rotate 270 CW
    6 = Rotate 90 CW
    7 = Mirror horizontal and rotate 90 CW
    8 = Rotate 270 CW
    

    In PIL
    FLIP_LEFT_RIGHT = 0
    FLIP_TOP_BOTTOM = 1
    ROTATE_180 = 3
    ROTATE_270 = 4
    ROTATE_90 = 2
    TRANSPOSE = 5
    TRANSVERSE = 6
    """
    # iamge.transpose comaptible dictionary
    # https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.transpose
    transpose_map = {
        1: None,                    # Horizontal (normal)
        2: Image.FLIP_LEFT_RIGHT,   # Mirror horizontal
        3: Image.ROTATE_180,        # Rotate 180
        4: Image.FLIP_TOP_BOTTOM,   # Mirror vertical
        5: Image.TRANSPOSE,         # Mirror horizontal and rotate 270 CW
        6: Image.ROTATE_270,        # Rotate 90 CW value of 4
        7: Image.TRANSVERSE,        # Mirror horizontal and rotate 90 CW
        8: Image.ROTATE_90          # Rotate 270 CW value of 2
    }
    
    with Image.open(img_path) as img:
        try:
            orientation = img._getexif().get(274, 0)
            # print(f"img_path: {img_path}, orientation: {orientation}")
        except AttributeError:
            return None
        
    return transpose_map[orientation]


In [7]:
exif_info = get_exif(r"Y:/datasets/dynamic_3DGS/data/basketball/ims/0/000000.jpg")
# exif_info = get_exif(all_jpgs[0])
if exif_info == {}:
    print("No EXIF data found for image")

for tag, value in exif_info.items():

    if tag == 'GPSInfo':
        print("GPSInfo")
        gps_info = get_exif_gps(exif_info)
        for tag, value in gps_info.items():
            print(f"{' ' * 5}{tag}: {value}")

    else:
        print(f"{tag}: {value}")
    

ExifImageWidth: 640
ExifImageHeight: 360


In [6]:
all_jpgs= glob.glob('/cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/data/basketball/ims/0/*.jpg')
print(f"total images: {len(all_jpgs)}")


parent_dir= os.path.dirname(all_jpgs[0])
print(f"parent directory: {parent_dir}")
print(f"parent directory: {os.path.basename(all_jpgs[0])}")


total images: 150
parent directory: /cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/data/basketball/ims/0
parent directory: 000054.jpg


In [7]:

# make a dataframe with 3 columns
camera_names_df= pd.DataFrame(columns=['tower', 'aim', 'name', 'focal_length', 'Width', 'Height', 'Camera_model', 'Orientation'])


for i, jpg in enumerate(all_jpgs):
    # camera_name= re.split('[-_]', os.path.basename(jpg))[2]
    camera_name= []
    exif_info = get_exif(jpg)
    
    # Create a dictionary with the new data
    try:
        camera_names_df.loc[len(camera_names_df)] = {
            'tower': camera_name[0] if len(camera_name) > 0 else None,
            'aim': camera_name[1] if len(camera_name) > 1 else None,
            'name': camera_name[2] if len(camera_name) > 2 else None,
            'focal_length': exif_info.get('FocalLength', None),
            'Width': exif_info.get('ExifImageWidth', None),
            'Height': exif_info.get('ExifImageHeight', None),
            'Camera_model': exif_info.get('LensModel', None),
            'Orientation': exif_info.get('Orientation', None)
        }
    except KeyError:
        print(f"KeyError: {i}, {jpg}")
        
    # if camera_name == 'KF1':
    #     print(i, exif_info['LensModel'])
    
# :NOTE change i to I
camera_names_df['tower']= camera_names_df['tower'].str.upper()


In [8]:
camera_names_df

,tower,aim,name,focal_length,Width,Height,Camera_model,Orientation
0,None,None,None,None,640,360,None,None
1,None,None,None,None,640,360,None,None
2,None,None,None,None,640,360,None,None
3,None,None,None,None,640,360,None,None
4,None,None,None,None,640,360,None,None
...,...,...,...,...,...,...,...,...
145,None,None,None,None,640,360,None,None
146,None,None,None,None,640,360,None,None
147,None,None,None,None,640,360,None,None
148,None,None,None,None,640,360,None,None


In [14]:
# camera_names_df.to_csv('camera_names.csv', index=False)

In [ ]:
# group by width, height, focal_length, tower, aim

group_df= pd.pivot_table(camera_names_df, index=['Width', 'Height', 'focal_length', 'tower', 'aim'], aggfunc='count')
display(group_df)

# Rotate each image and change exif tag

In [15]:
transpose_map = {
        1: None,                    # Horizontal (normal)
        2: Image.FLIP_LEFT_RIGHT,   # Mirror horizontal
        3: Image.ROTATE_180,        # Rotate 180
        4: Image.FLIP_TOP_BOTTOM,   # Mirror vertical
        5: Image.TRANSPOSE,         # Mirror horizontal and rotate 270 CW
        6: Image.ROTATE_270,        # Rotate 90 CW value of 4
        7: Image.TRANSVERSE,        # Mirror horizontal and rotate 90 CW
        8: Image.ROTATE_90          # Rotate 270 CW value of 2
    }


for jpg in tqdm(all_jpgs):
    with Image.open(jpg) as img:
        
        # exif_info = get_exif(jpg)
        # print(exif_info)
        exif_dict = piexif.load(img.info['exif'])
        # print(exif_dict)
        # orientation = get_exif_orientation(jpg)
        orientation= exif_dict.get('0th', {}).get(274, 1)
        # print(orientation, img._getexif().get(274, 0))

        if orientation != 1:
            orientation= transpose_map[orientation]
            exif_dict['0th'][274]= 1
        #     # print(exif_dict)

        #     with Image.open(jpg) as img:
            img= img.transpose(orientation)

                

            exif_bytes = piexif.dump(exif_dict)
            base, ext = os.path.splitext(jpg)
            new_fname = f"{base}{ext}"
            # print(new_fname)
            img.save(new_fname, "jpeg", exif=exif_bytes)

    #             # img.save(jpg)
    #             # print(f"orientation: {orientation}, img_path: {jpg}")
    #     # break


100%|██████████| 92/92 [00:25<00:00,  3.61it/s]


# Semgnent each image based on the mask

In [26]:
all_masks= glob.glob('/cs/student/projects4/ml/2023/asrivast/datasets/handbag_19/visible_model/mask_upright/*.JPG.png')
# all_jps is at /cs/student/projects4/ml/2023/asrivast/datasets/handbag_19/visible_model/segmented_upright/*.JPG
for mask,jpg in tqdm(zip(all_masks, all_jpgs)):
    with Image.open(jpg) as img:
        with Image.open(mask) as mask_img:
            # mask img based on mask_img
            # the mask is inverted
            mask_img= mask_img.convert('L')

            # Invert the mask
            # inverted_mask = ImageOps.invert(mask_img)

            # Create a black background image
            black_background = Image.new('RGB', img.size, (0, 0, 0))
            
            # Paste the original image onto the black background using the inverted mask
            black_background.paste(img, (0, 0), mask_img)

            # Save the modified image
            base, ext = os.path.splitext(jpg)
            new_fname = f"{base}{ext}"
            black_background.save(new_fname)

    # break

    
    # print(mask, new_fname)
    # break

50it [00:14,  3.37it/s]


# make custom match text file

parameters to take
* focal length
* no. of towers on either side to consider

In [37]:
from typing import List
def n_closest_towers(tower, n:int= 1,)-> List[str]:
    """
    This function returns a list of n towers closest to the given tower in a clockwise direction.
    The tower names are represented by capital letters from 'A' to '
    """
    
    clock_tower_ring= {"A":"B", "B":"C", "C":"D", "D":"E", "E":"F", "F":"G", "G":"H", "H":"i", "i":"J", "J":"K","K":"A"}

    next_tower= tower
    tower_list= [next_tower]
    for i in range(n):
        next_tower= clock_tower_ring[next_tower]
        tower_list.append(next_tower)
    
    anticlockwise_tower_ring = {value: key for key, value in clock_tower_ring.items()}
    next_tower= tower
    for i in range(n):
        next_tower= anticlockwise_tower_ring[next_tower]
        tower_list.append(next_tower)
    
    return tower_list

# simple tests for n_closest_towers function 
print(n_closest_towers('A', 1))
print(n_closest_towers('B', 3))
print(n_closest_towers('H', 2))


['A', 'B', 'K']
['B', 'C', 'D', 'E', 'A', 'K', 'J']
['H', 'i', 'J', 'G', 'F']


In [ ]:
a= re.split("[._-]",os.path.basename(all_jpgs[0]))
# b= a
b= a[-4]+"_"+a[-3]+"_"+a[-2]+"."+a[-1]
b

In [39]:
focal_length= str(float(50))
num_towers= 1

parent_dir= "C:/Users/student1/Desktop/Datasets/dress2_10"
all_jpgs= glob.glob(f"{parent_dir}/focal_length_2/{focal_length}/*.JPG")
scene_name= os.path.basename(all_jpgs[0])[13: -4]
print(f"total images: {len(all_jpgs)} in scene: {scene_name}")
print(all_jpgs[0])

total images: 70 in scene: dress2_sunglasses_10
C:/Users/student1/Desktop/Datasets/dress2_10/focal_length_2/50.0\marie_PL-AB1_dress2_sunglasses_10.JPG


In [41]:
all_jpgs= glob.glob(f"{parent_dir}/focal_length_2/{focal_length}/marie_*-A*_{scene_name}.JPG")
print(f"total images: {len(all_jpgs)}")
all_jpgs

total images: 7


['C:/Users/student1/Desktop/Datasets/dress2_10/focal_length_2/50.0\\marie_PL-AB1_dress2_sunglasses_10.JPG',
 'C:/Users/student1/Desktop/Datasets/dress2_10/focal_length_2/50.0\\marie_PL-AB2_dress2_sunglasses_10.JPG',
 'C:/Users/student1/Desktop/Datasets/dress2_10/focal_length_2/50.0\\marie_PL-AB3_dress2_sunglasses_10.JPG',
 'C:/Users/student1/Desktop/Datasets/dress2_10/focal_length_2/50.0\\marie_PL-AB4_dress2_sunglasses_10.JPG',
 'C:/Users/student1/Desktop/Datasets/dress2_10/focal_length_2/50.0\\marie_PL-AB5_dress2_sunglasses_10.JPG',
 'C:/Users/student1/Desktop/Datasets/dress2_10/focal_length_2/50.0\\marie_PL-AB7_dress2_sunglasses_10.JPG',
 'C:/Users/student1/Desktop/Datasets/dress2_10/focal_length_2/50.0\\marie_PL-ABM_dress2_sunglasses_10.JPG']

In [42]:
os.makedirs(f"{parent_dir}/COLMAP_models/focal_length_2/{focal_length}", exist_ok=True)
txt_path= f"{parent_dir}/COLMAP_models/focal_length_2/{focal_length}/tower_{num_towers}_clock.txt"

all_jpgs= glob.glob(f"{parent_dir}/focal_length_2/{focal_length}/*.JPG")
# get all jpgs with the same focal length

with open(txt_path, 'w') as f:
    for i, jpg in enumerate(all_jpgs):
        camera_name= re.split('[-_]', os.path.basename(jpg))[2]
        tower= camera_name[0]
        n_closest_towers_list= n_closest_towers(tower, num_towers)
        for tower in n_closest_towers_list:
            # list of all jpgs with structure like this
            nearest_tower_jpgs= glob.glob(f"{parent_dir}/focal_length_2/{focal_length}/marie_*-{tower}*_{scene_name}.JPG")
            for near_tower_jpg in nearest_tower_jpgs:
                f.write(f"{os.path.basename(jpg)} {os.path.basename(near_tower_jpg)}\n")

# print done
print(f"Done writing to {txt_path}")

Done writing to C:/Users/student1/Desktop/Datasets/dress2_10/COLMAP_models/focal_length_2/50.0/tower_1_clock.txt


In [7]:

# open png image
from PIL import Image


all_pngs= glob.glob('/cs/student/projects4/ml/2023/asrivast/datasets/handbag_19/visible_model/segmented/*.png')
all_jpgs= glob.glob('/cs/student/projects4/ml/2023/asrivast/datasets/handbag_19/visible_model/images/*.JPG')

print(f"total images: {len(all_pngs)}")
parent_dir= os.path.dirname(os.path.dirname(all_pngs[0]))
mask_path= os.path.join(parent_dir, 'mask_unrotated')
os.makedirs(mask_path, exist_ok=True)

for jpg, png in tqdm(zip(all_jpgs, all_pngs)):
    
    with Image.open(png) as img:
        img_name= os.path.basename(png).replace('png', 'JPG.png')
        np_img= np.asarray(img)

        alpha_channel= np_img[:,:,3]
        # Create a mask where transparent pixels are black (0) and non-transparent pixels are white (255)
        mask = np.where(alpha_channel > 0, 255, 0).astype(np.uint8)

        # Convert the mask to an image
        mask_img = Image.fromarray(mask, mode='L') # mode L is for 8-bit pixels, black and white
        rotation= get_exif_orientation(jpg)
        # print(type(rotation))
        # rotate the image based on the orientation
        # mask_img= mask_img.transpose(rotation)
        # print(f"{img.size} {mask_img.size}")

        mask_img.save(f"{mask_path}/{img_name}")
        # break

total images: 50


50it [00:25,  1.95it/s]


In [21]:
from tqdm import tqdm

# get all jpg in folder and sub folders at this directory

parent_dir= "C:/Users/student1/Desktop/Datasets/dress2_10/focal_length"
all_jpgs= glob.glob(f"{parent_dir}/**/*.JPG", recursive=True)
print(f"total images: {len(all_jpgs)}")

# nowmoveall imagesto this folder andmake it if it does not exist
new_folder= "C:/Users/student1/Desktop/Datasets/dress2_10/visible_model"
os.makedirs(new_folder, exist_ok=True)
for jpg in tqdm(all_jpgs):
    shutil.copy2(jpg, new_folder)
    # break

total images: 139


  0%|          | 0/139 [00:00<?, ?it/s]

100%|██████████| 139/139 [00:03<00:00, 34.99it/s]


In [8]:
parent_dir= "C:/Users/student1/Desktop/Datasets/handbag_19/visible_model"
all_jpgs= glob.glob(f"{parent_dir}/*.JPG", recursive=True)
print(f"total images: {len(all_jpgs)}")

# move to new folder and resize
dest_folder= "C:/Users/student1/Desktop/Datasets/handbag_19/visible_model_4"
os.makedirs(dest_folder, exist_ok=True)
for jpg in tqdm(all_jpgs):
    dest_file= os.path.join(dest_folder, os.path.basename(jpg))
# #     # dest_file= jpg
    shutil.copy2(jpg, dest_file)
    exit_code = os.system("magick" + f" mogrify -resize 25% " + dest_file)
#     # print(dest_file, exit_code)
    # break


total images: 50


100%|██████████| 50/50 [00:25<00:00,  1.98it/s]


In [6]:
100/75

1.3333333333333333

In [33]:
all_jpgs[0]

'C:/Users/student1/Desktop/Datasets/dress2_10/focal_length_2\\122.0\\marie_PL-GH2_dress2_sunglasses_10.JPG'

# Ship sparse folder and images to drive

In [76]:
parent_dir= "C:/Users/student1/Desktop/Datasets/dress2_10"
scene_name= os.path.basename(parent_dir)
experiment_name= "tower_ABCD_2"


dest_folder= "Y:\datasets"
dest_folder= os.path.join(dest_folder, scene_name, experiment_name)
os.makedirs(dest_folder, exist_ok=True)
os.makedirs(os.path.join(dest_folder, "images"), exist_ok=True)

all_jpgs= glob.glob(f"{parent_dir}/{experiment_name}/*.JPG", recursive=True)
print(f"total images: {len(all_jpgs)}")
for jpg in tqdm(all_jpgs):
    dest_file= os.path.join(dest_folder,"images", os.path.basename(jpg))
    shutil.copy2(jpg, dest_file)
    # break

# move sparse model folder
spare_folder_src= os.path.join(parent_dir, "COLMAP_models", experiment_name, "sparse")
# check if its there
if os.path.exists(spare_folder_src):
    spare_folder_dest= os.path.join(dest_folder, "sparse")
    # if os.path.exists(spare_folder_dest):
    #     shutil.rmtree(spare_folder_dest)
    try:
        shutil.copytree(spare_folder_src, spare_folder_dest)
    except FileExistsError:
        print(f"{spare_folder_dest} already exists")
else:
    print(f"{spare_folder_src} does not exist")

total images: 35


  0%|          | 0/35 [00:00<?, ?it/s]

100%|██████████| 35/35 [00:01<00:00, 32.31it/s]


# Now move the results folder back

In [70]:
results_folder_src= os.path.join(dest_folder, "results")

results_folder_dest= os.path.join(parent_dir, "3DGS_models", experiment_name)

if os.path.exists(results_folder_src):
    print(f"{results_folder_src} exists")
    try:
        shutil.copytree(results_folder_src, results_folder_dest)
    except FileExistsError:
        print(f"{results_folder_dest} already exists")

Y:\datasets\dress3_15\visible_model_2\results exists


In [69]:
results_folder_dest

'C:/Users/student1/Desktop/Datasets/dress3_15\\dress3_15\\3DGS_models\\visible_model_2'

# Video dataset

In [2]:
all_jpgs= glob.glob('Y:\datasets\portsmouth_video\cameras\**\*.JPG')
print(f"total images: {len(all_jpgs)} per camera {len(all_jpgs)/32}")

total images: 35520 per camera 1110.0


In [32]:
cropped= {}

In [42]:
img.close()

In [3]:
new_folder= os.path.join("Y:\datasets\portsmouth_video", "ims")
# make new folder if it does not exist
os.makedirs(new_folder, exist_ok=True)
for jpg in tqdm(all_jpgs):
    camera_num= int(re.split('[-_.]', os.path.basename(jpg))[1])
    frame_num= re.split('[-_.]', os.path.basename(jpg))[2]
    
    if int(frame_num) < 200:
        dest_folder= os.path.join(new_folder, f"{camera_num}")
        os.makedirs(dest_folder, exist_ok=True)
        dest_file= os.path.join(dest_folder, f"{frame_num}.JPG")
        shutil.copy2(jpg, dest_file)
        # crop the bottom 2 pixels

        with Image.open(dest_file) as img:
            width, height = img.size
            
            cropped_img= img.crop((0, 0, width, height-30))
            cropped_img.save(dest_file)
        # break
    # move to a foler camera_num/frame_num

100%|██████████| 35520/35520 [03:59<00:00, 148.16it/s]
